# Creating a Grid of PSFs in WebbPSF with `psf_grid()`

In order to use photutils to do PSF-fitting photometry on simulated JWST PSFs, the first step is to create a grid of fiducial PSFs spanning the instrument/detector of choice. This can be done using the `psf_grid()` method in WebbPSF which will output a (list of or single) photutils `GriddedPSFModel` object(s). Users can then use photutils to apply interpolation to the grid to simulate a spatially dependent PSF anywhere on the instrument.

This notebook will describe how to use WebbPSF to create these grids of PSFs. 

<img src="example_psf_grid.png" width=350px>

<div class="alert alert-warning">Warning: This notebook creates many large GriddedPSFModel objects and will therefore take a long time to run. We suggest only running one or two examples as the examples just show the different parameters available when creating the grid of PSFs.</div>

For those who have used WebbPSF before, `psf_grid()` is set up in a similar manner to `calc_psf()`. A basic use of this method looks like:

In [ ]:
from astropy.io import fits
import webbpsf
from webbpsf.utils import to_griddedpsfmodel
print("Notebook tested with WebbPSF 0.8.0, currently running on WebbPSF", webbpsf.version.version)

In [ ]:
# Create a grid of PSFs for all detectors in NIRCam
nir = webbpsf.NIRCam()
nir.filter = "F090W"
list_of_grids = nir.psf_grid()

### Outputs

For each instrument/filter/detector combination, `psf_grid()` will return a 3D array of Z $\times$ n $\times$ n in a `GriddedPSFModel` object. The 3D array represents the Z number of 2D n $\times$ n PSFs created on the detector. You can check the `grid_xypos` key (or if you save the object as a `FITS` file, the `DET_YX#` header keywords) which will state the position of the PSF on the detector for each of the PSFs. The order of the tuples in `grid_xypos` (and the number in the `FITS` header keyword) refers to the number the PSF is in the 3D array.

If you set `all_detectors=True` and the instrument you chose has more than one detector (e.g. NIRCam), `psf_grid()` will return a list of 3D `GriddedPSFModel` objects, with one object per detector. If you defined only 1 detector, or have `all_detectors=True` when your instrument only has 1 detector (e.g. NIRISS), `psf_grid()` will return a single `GriddedPSFModel` object.

If you set `save=True`, the `GriddedPSFModel` will be saved as a `FITS` object. This can later be read in and converted back to a `GriddedPSFModel`. The method for how to do this will be detailed below.

### Summary of Possible Parameters

    Parameters to be set before psf_grid()
    --------------------------------------
    Filter(s)
        Set with: inst.filter = str
        If you want to run all the filters, you must loop through them
    Detector(s)
        Set with: inst.detector = str
        If you want to run 1 detector, define it like above and set the internal parameter all_detectors=False
        If you want to run a list of detectors, you must loop through them and set all_detectors=False
        If you want to run all the detectors, you set all_detectors=True
    PSF location if num_psfs=1 
        Set with: inst.detector_position = tuple in the form (x,y)
        Or if you want the location to be the center of the detector, set single_psf_centered=True
    Other WebbPSF attributes
        Set with the instrument attribute. E.g. inst.pupil_mask, inst.image_mask, inst.pupilopd, etc
    
    
    Parameters inside psf_grid()
    ----------------------------
    all_detectors: bool
        If True, run all detectors for the instrument (For NIRCam, this will pick out either all 
        the shortwave or all the longwave detectors). If False, run for the detector set in
        the instrument instance. Default is True
    use_detsampled_psf: bool
        If True, the grid of PSFs returned will be detector sampled (made by binning down the
        oversampled PSF). If False, the PSFs will be oversampled by the factor defined by the
        oversample/detector_oversample/fft_oversample keywords. Default is False.
    num_psfs: int
        The total number of fiducial PSFs to be created and saved in the files. This
        number must be a square number. Default is 16.
        E.g. num_psfs = 16 will create a 4x4 grid of fiducial PSFs.
    single_psf_centered: bool
        If num_psfs is set to 1, this defines where that psf is located. If True it will be the
        center of the detector, if False it will be the location defined in the WebbPSF
        attribute detector_position (reminder - detector_position is (x,y)). Default is True.
    save: bool
        True/False boolean if you want to save your file. Default is False.
    outfile: str
        If "save" keyword is set to True, your current file will be saved under
        "{outfile}_det_filt.fits". Default of None will save it in the current
        directory as: instr_det_filt_fovp#_samp#_npsf#.fits
    overwrite : bool
        True/False boolean to overwrite the output file if it already exists. Default
        is True.
    verbose : bool
        True/False boolean to print status updates. Default is True.
    **kwargs:
        Add any extra arguments to the WebbPSF calc_psf() method call. This includes
        the following kwargs (and their default values): source(=None), nlambda(=None),
        monochromatic(=None), fov_arcsec(=None), fov_pixels(=101), oversample(=4),
        detector_oversample(=None), fft_oversample(=None), normalize(='first'),
        add_distortion(=True), and crop_psf(=True)


## Example Uses

### Create a full library of grids for 1 filter and all detectors

This is the same example as above

In [ ]:
nir = webbpsf.NIRCam()
nir.filter = "F090W"
grid = nir.psf_grid()

In [ ]:
# Check out the data and meta data with
nrca1_model = grid[0]
print(nrca1_model.data)
print(nrca1_model.meta)

### Create a grid for 1 filter and 1 detector

In [ ]:
nir = webbpsf.NIRCam()
nir.filter = "F090W"
nir.detector = "NRCA2"
grid = nir.psf_grid(all_detectors=False)

### Loop through if you want multiple filters or a subset of the detectors

In [ ]:
# Loop through filters
grid_filt_list = []
nis = webbpsf.NIRISS()
for filt in nis.filter_list:
    nis.filter = filt
    grid = nis.psf_grid()
    grid_filt_list.append(grid)

# Loop through detectors
nir = webbpsf.NIRCam()
nir.filter = "F140M"
grid_det_list = []
for det in ["NRCA1", "NRCA2", "NRCA3", "NRCA4"]:
    nir.detector = det
    grid = nir.psf_grid(all_detectors=False)  
    grid_det_list.append(grid)

### Set the number of PSFs

Use the `num_psfs` keyword to set the number of PSFs in the grid. `num_psfs = 4` means you'll have a 2x2 grid. Default is 16

In [ ]:
nis = webbpsf.NIRISS()
nis.filter = "F140M"
grid = nis.psf_grid(num_psfs=4)

### Include any `calc_psf()` **kwargs in `psf_grid()`

See the docstring of `calc_psf()` for a full description of these keyword arguments

In [ ]:
fgs = webbpsf.FGS()
fgs.filter = "FGS"
grid = fgs.psf_grid(fov_pixels=1000, oversample=3, add_distortion=True, nlambda=30)

### Customize with other WebbPSF settings before the `psf_grid()` method call

Anything you can set in WebbPSF can be applied here. Say you want to use a different OPD, your own OPD with extra mirror moves applied to it, a pupil/image mask combination, etc:

In [ ]:
nir = webbpsf.NIRCam()
nir.filter = "F090W"

# Setting extra aspects you want applied to all the PSFs generated in the grid:
nir.pupilopd = ('OPD_RevW_ote_for_NIRCam_predicted.fits.gz', 7)
nir.pupil_mask = "WEAK LENS +4"

grid = nir.psf_grid()

### Turn off status updates with `verbose=False`

In [ ]:
mir = webbpsf.MIRI()
mir.filter = "F560W"
grid = mir.psf_grid(verbose=False)

### Saving files

Alongside returning the PSF grid, users can save the `GriddedPSFModel` objects as `FITS` files to create their own library. 

Users can set `save=True` and then have 2 options for the naming of these files. They can leave `outfile=None` to use the default naming scheme which will save the files in the current directory. Otherwise they can set `outfile` to be a path and the beginning of the filename. The fiename will then have the detector and filter name appended to it and will be saved in the specified path.

In [ ]:
nir = webbpsf.NIRCam()
nir.filter = "F090W"

# Don't save (Default)
grid = nir.psf_grid()

# Save with the default name -> {instr}_{det}_{filt}_fovp{#}_samp{#}_npsf{#}.fits
grid = nir.psf_grid(save=True, outfile=None, overwrite=True)

To convert the `FITS` file back to a `GriddedPSFModel` object, use `to_griddedpsfmodel()`. You can pass this either a filename or an HDUlist object.

In [ ]:
from webbpsf.utils import to_griddedpsfmodel

# As a file
grid = to_griddedpsfmodel("nircam_nrca1_f090w_fovp101_samp4_npsf16.fits")  # file created 2 cells above

# As an HDUlist object
hdu = fits.open("nircam_nrca1_f090w_fovp101_samp4_npsf16.fits")
grid = to_griddedpsfmodel(hdu, ext=0)

## Not Recommended but Additional Options

### Detector-sampled PSF

If you want your ouput grid of PSFs to be detector sampled, set `use_detsampled_psf=True`

In [ ]:
nir = webbpsf.NIRCam()
nir.filter = "F090W"
grid = nir.psf_grid(use_detsampled_psf=True)

### If `num_psfs=1`, set the location of the PSF with detector_position attribute

If `single_psf_centered = True`, then the single PSF will be at the center of the detector. If it's `False`, then the location of the PSF is pulled from the `detector_position` instrument attribute

In [ ]:
# Option 1: Set the location as the center of the detector (Default)
nis = webbpsf.NIRISS()
nis.filter = "F140M"
grid = nis.psf_grid(num_psfs=1)

# Option 2: Choose your own location
nis = webbpsf.NIRISS()
nis.filter = "F140M"
nis.detector_position = (10,10) # (x,y)
grid = nis.psf_grid(num_psfs=1, single_psf_centered=False)